In [1]:
import sys, os
sys.path.append('..')

# Verify the path is correct
print(f"Current directory: {os.getcwd()}")
print(f"Parent directory exists: {os.path.exists('..')}")
print(f"Utils directory exists: {os.path.exists('../utils')}")

from utils.prompts import render
from utils.llm_client import LLMClient #pick llm client
from utils.logging_utils import log_llm_call #log every api call
from utils.router import  pick_model, should_use_reasoning_model #general, reasoning, strrong



Current directory: c:\Users\viraj\Zuu\AI_Engineer_Bootcamp\notebooks
Parent directory exists: True
Utils directory exists: True


In [2]:
os.getcwd()

'c:\\Users\\viraj\\Zuu\\AI_Engineer_Bootcamp\\notebooks'

### 01- Zero Shot

In [ ]:
prompt_text , spec = render(
    "zero_shot.v1",
    role = "sentiment_analysis",
    instructions = "Analyze the following text and determine the sentiment as positive/negative/neutral",
    constrains = "the sentiment should be one of the following: positive, negative, neutral",
    format = "the sentiment is : {sentiment}"
    )


model = pick_model("groq", "general")
llm = LLMClient("groq", model)

text = "im really happy with the service i received today!"
messages=[
        {"role": "user", 
        "content": f"{prompt_text} Review: {text}"
        }
    ]

llm.chat(messages, temperature=0.2)

{'text': '${instruction}: Analyze the sentiment of the given text\n${constraints}: None\nthe sentiment is : Negative',
 'usage': {'input_tokens_est': 48,
  'context_tokens_est': 0,
  'total_est': 51,
  'prompt_tokens_actual': 79,
  'completion_tokens_actual': 23,
  'total_tokens_actual': 102},
 'latency_ms': 549,
 'raw': ChatCompletion(id='chatcmpl-76d1a824-6260-43a5-9271-4456adc5c844', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='${instruction}: Analyze the sentiment of the given text\n${constraints}: None\nthe sentiment is : Negative', role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1769918181, model='llama-3.1-8b-instant', object='chat.completion', mcp_list_tools=None, service_tier='on_demand', system_fingerprint='fp_1151d4f23c', usage=CompletionUsage(completion_tokens=23, prompt_tokens=79, total_tokens=102, completion_time=0.046389084, completion_tokens_d